In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose

In [10]:
# changing the format of the given csv file
file_path = "sales_data_storytelling.csv"  

with open(file_path, 'r') as file:
    content = file.read()

updated_content = content.replace(';', ',')

with open(file_path, 'w') as file:
    file.write(updated_content)

In [ ]:
df = pd.read_csv(file_path)


df['Month'] = pd.to_datetime(df['Month'])
df['MonthName'] = df['Month'].dt.strftime('%b %Y')


monthly_revenue = df.groupby(['Month', 'MonthName'])['Revenue'].sum().reset_index()
monthly_revenue = monthly_revenue.sort_values('Month')

# Identify peak and low months
peak_month = monthly_revenue.loc[monthly_revenue['Revenue'].idxmax()]
low_month = monthly_revenue.loc[monthly_revenue['Revenue'].idxmin()]

# Plotting
plt.figure(figsize=(12, 6))
sns.set_style("whitegrid")

ax = sns.barplot(data=monthly_revenue, x='MonthName', y='Revenue', color='steelblue')

# Highlight peak and low months
ax.patches[monthly_revenue['Revenue'].idxmax()].set_facecolor('green')
ax.patches[monthly_revenue['Revenue'].idxmin()].set_facecolor('red')

# Annotations
plt.text(monthly_revenue['Revenue'].idxmax(), peak_month['Revenue']+500, 
         f"Peak: {peak_month['MonthName']}\n${peak_month['Revenue']:,.2f}", ha='center', color='green')
plt.text(monthly_revenue['Revenue'].idxmin(), low_month['Revenue']+500, 
         f"Low: {low_month['MonthName']}\n${low_month['Revenue']:,.2f}",  ha='center', color='red')

# Formatting
plt.title('Monthly Revenue Trends (2023)', fontsize=14, pad=20)
plt.xlabel('Month', labelpad=10)
plt.ylabel('Total Revenue ($)', labelpad=10)
plt.xticks(rotation=45)
plt.tight_layout()

ax.yaxis.set_major_formatter('${x:,.0f}')

plt.show()

In [ ]:


# Calculate average revenue per product
product_avg_revenue = df.groupby('Product')['Revenue'].mean().sort_values(ascending=False).reset_index()

# Calculate total units sold per product (to identify best-selling item)
product_units_sold = df.groupby('Product')['UnitsSold'].sum().sort_values(ascending=False).reset_index()
best_selling_product = product_units_sold.iloc[0]


plt.figure(figsize=(12, 6))
sns.set_style("whitegrid")

# Bar plot for average revenue
ax = sns.barplot(data=product_avg_revenue, x='Product', y='Revenue', palette='Blues_d')

# Highlight the bar with highest average revenue
max_rev_product = product_avg_revenue.iloc[0]
ax.patches[0].set_facecolor('darkblue')

# Add annotations
plt.text(0, max_rev_product['Revenue']+100, f"Highest Avg: {max_rev_product['Product']}\n${max_rev_product['Revenue']:,.2f}", ha='center', color='darkblue', weight='bold')

# Formatting
plt.title(f'Average Revenue by Product (2023)\nBest-Selling Item: {best_selling_product["Product"]} ({best_selling_product["UnitsSold"]} units)', fontsize=14, pad=20)
plt.xlabel('Product', labelpad=10)
plt.ylabel('Average Monthly Revenue ($)', labelpad=10)
plt.xticks(rotation=45)

# Format y-axis as currency
ax.yaxis.set_major_formatter('${x:,.0f}')

# Display best-selling item info
plt.text(0.5, -0.3, 
         f"Best-Selling Product by Units Sold: {best_selling_product['Product']} ({best_selling_product['UnitsSold']} units total)",
         ha='center', va='center', transform=ax.transAxes, fontsize=12, bbox=dict(facecolor='lightgray', alpha=0.5))

plt.tight_layout()
plt.show()

In [ ]:
# Calculate total revenue by product
product_revenue = df.groupby('Product')['Revenue'].sum().sort_values(ascending=False)

# Calculate percentage contribution
total_revenue = product_revenue.sum()
product_percentage = (product_revenue / total_revenue * 100).round(1)

# Create figure with two subplots
plt.figure(figsize=(16, 6))
sns.set_style("whitegrid")

plt.subplot(1, 2, 1)
colors = sns.color_palette('pastel')
patches, texts, autotexts = plt.pie(product_revenue, labels=product_revenue.index, colors=colors, autopct='%1.1f%%', startangle=90, wedgeprops={'linewidth': 1, 'edgecolor': 'white'})

# Improve pie chart labels
plt.setp(autotexts, size=10, weight="bold", color='black')
plt.setp(texts, size=10)
plt.title('Product Revenue Contribution (Pie Chart)', pad=20)


In [ ]:

df['Month'] = pd.to_datetime(df['Month'])
monthly_sales = df.groupby([df['Month'].dt.strftime('%Y-%m'), 'Product'])['Revenue'].sum().unstack()

## 1. Seasonal Decomposition for Top Products
plt.figure(figsize=(14, 8))
for i, product in enumerate(monthly_sales.columns[:3]):  # Analyze top 3 products
    result = seasonal_decompose(monthly_sales[product], model='additive', period=3)
    
    plt.subplot(3, 3, i*3+1)
    result.trend.plot(title=f'{product} - Trend', color='blue')
    plt.ylabel('Revenue')
    
    plt.subplot(3, 3, i*3+2)
    result.seasonal.plot(title=f'{product} - Seasonality', color='green')
    
    plt.subplot(3, 3, i*3+3)
    result.resid.plot(title=f'{product} - Residuals', color='red')
    plt.ylabel('Deviation')

plt.tight_layout()
plt.suptitle('Seasonal Decomposition of Top Products', y=1.02)
plt.show()

## 2. Heatmap of Monthly Product Performance
plt.figure(figsize=(12, 6))
monthly_pivot = df.pivot_table(values='Revenue', index=df['Month'].dt.month_name(), columns='Product', aggfunc='sum')
sns.heatmap(monthly_pivot, cmap='YlGnBu', annot=True, fmt=".0f", linewidths=.5)
plt.title('Monthly Revenue Heatmap by Product', pad=20)
plt.xlabel('Product')
plt.ylabel('Month')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 3. Anomaly Detection Using Z-Scores
plt.figure(figsize=(14, 6))
for product in monthly_sales.columns:
    product_data = monthly_sales[product]
    z_scores = (product_data - product_data.mean()) / product_data.std()
    anomalies = z_scores.abs() > 2
    
    plt.scatter(product_data.index, product_data, 
                c=anomalies.map({True: 'red', False: 'blue'}),
                label=product)
    plt.plot(product_data.index, product_data, alpha=0.3)

plt.title('Monthly Revenue with Anomalies Highlighted (Z-score > 2)', pad=20)
plt.xlabel('Month')
plt.ylabel('Revenue ($)')
plt.xticks(rotation=45)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()